In [1]:
import os
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import LeaveOneOut

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids

from jupyterthemes import jtplot
# change plotting style to match the theme
jtplot.style()


%matplotlib inline

/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
accidents_data = pd.DataFrame()
for i in os.listdir():
    if 'Accidents' in i:
        accidents_data = accidents_data.append(pd.read_csv(i, low_memory=False))

accidents_data['Date'] = pd.to_datetime(accidents_data['Date'])
accidents_data = accidents_data[accidents_data['Date'] > '01/01/2012']

# Remove entries with missing coordinates
accidents_data.dropna(subset = ['Longitude','Latitude'], inplace=True)

accidents_data['hours'] = accidents_data['Time'].apply(lambda x: int(str(x)[:2]) if pd.notnull(x) else -1)
accidents_data['Month'] = accidents_data['Date'].apply(lambda x: x.month)

# KDE

In [ ]:
from sklearn.neighbors import KernelDensity
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import LeaveOneOut

acc_vals = accidents_data[['Latitude', 'Longitude']].values

print("Spherical Coord KDE")
kde = KernelDensity(bandwidth=1., metric='haversine', kernel='gaussian', algorithm='ball_tree')
kde.fit(np.radians(acc_vals))

np.exp(kde.score_samples(acc_vals))

# score_samples returns the log of the probability density
#logprob = kde.score_samples(x_d[:, None])

#plt.fill_between(x_d, np.exp(logprob), alpha=0.5)
#plt.plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)
#plt.ylim(-0.02, 0.22)

xgrid = accidents_data['Latitude'].values
ygrid = accidents_data['Longitude'].values

mesh_x, mesh_y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])

stack_xy = np.radians(np.vstack([mesh_y.ravel(), mesh_x.ravel()]).T)

bandwidths = np.linspace(0.1, 10, 100)
grid = GridSearchCV(KernelDensity(kernel='gaussian'), {'bandwidth': bandwidths}, cv=LeaveOneOut(len(acc_vals)))
grid.fit(acc_vals)

print('Best bandwidth: ', grid.best_params_)

In [ ]:
from mpl_toolkits.basemap import Basemap
from sklearn.datasets.species_distributions import construct_grids
from sklearn.datasets import fetch_species_distributions

data = fetch_species_distributions()

# Get matrices/arrays of species IDs and locations
latlon = np.vstack([data.train['dd lat'],
                    data.train['dd long']]).T
species = np.array([d.decode('ascii').startswith('micro')
                    for d in data.train['species']], dtype='int')


xgrid, ygrid = construct_grids(data)

# Set up the data grid for the contour plot
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()
xy = np.vstack([Y.ravel(), X.ravel()]).T
xy = np.radians(xy[land_mask])

# Create two side-by-side plots
fig, ax = plt.subplots(1, 2)
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']
cmaps = ['Purples', 'Reds']

for i, axi in enumerate(ax):
    axi.set_title(species_names[i])
    
    # plot coastlines with basemap
    m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                urcrnrlat=Y.max(), llcrnrlon=X.min(),
                urcrnrlon=X.max(), resolution='c', ax=axi)
    m.drawmapboundary(fill_color='#DDEEFF')
    m.drawcoastlines()
    m.drawcountries()
    
    # construct a spherical kernel density estimate of the distribution
    kde = KernelDensity(bandwidth=0.03, metric='haversine')
    kde.fit(np.radians(latlon[species == i]))

    # evaluate only on the land: -9999 indicates ocean
    Z = np.full(land_mask.shape[0], -9999.0)
    Z[land_mask] = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    axi.contourf(X, Y, Z, levels=levels, cmap=cmaps[i])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids
from sklearn.neighbors import KernelDensity

# if basemap is available, we'll use it.
# otherwise, we'll improvise later...
try:
    from mpl_toolkits.basemap import Basemap
    basemap = True
except ImportError:
    basemap = False

# Get matrices/arrays of species IDs and locations
data = fetch_species_distributions()
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']

Xtrain = np.vstack([data['train']['dd lat'],
                    data['train']['dd long']]).T
ytrain = np.array([d.decode('ascii').startswith('micro')
                  for d in data['train']['species']], dtype='int')
Xtrain *= np.pi / 180.  # Convert lat/long to radians

# Set up the data grid for the contour plot
xgrid, ygrid = construct_grids(data)
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()

xy = np.vstack([Y.ravel(), X.ravel()]).T
xy = xy[land_mask]
xy *= np.pi / 180.

# Plot map of South America with distributions of each species
fig = plt.figure()
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)

for i in range(2):
    plt.subplot(1, 2, i + 1)

    # construct a kernel density estimate of the distribution
    print(" - computing KDE in spherical coordinates")
    kde = KernelDensity(bandwidth=0.04, metric='haversine',
                        kernel='gaussian', algorithm='ball_tree')
    kde.fit(Xtrain[ytrain == i])

    # evaluate only on the land: -9999 indicates ocean
    Z = -9999 + np.zeros(land_mask.shape[0])
    Z[land_mask] = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)

    if basemap:
        print(" - plot coastlines using basemap")
        m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                    urcrnrlat=Y.max(), llcrnrlon=X.min(),
                    urcrnrlon=X.max(), resolution='c')
        m.drawcoastlines()
        m.drawcountries()
    else:
        print(" - plot coastlines from coverage")
        plt.contour(X, Y, land_reference,
                    levels=[-9999], colors="k",
                    linestyles="solid")
        plt.xticks([])
        plt.yticks([])

    plt.title(species_names[i])

plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
from sklearn.datasets.species_distributions import construct_grids
from sklearn.datasets import fetch_species_distributions

data = fetch_species_distributions()

# Get matrices/arrays of species IDs and locations
latlon = np.vstack([data.train['dd lat'],
                    data.train['dd long']]).T
species = np.array([d.decode('ascii').startswith('micro')
                    for d in data.train['species']], dtype='int')


xgrid, ygrid = construct_grids(data)

# Set up the data grid for the contour plot
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()
xy = np.vstack([Y.ravel(), X.ravel()]).T
xy = np.radians(xy[land_mask])

# Create two side-by-side plots
fig, ax = plt.subplots(1, 2)
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']
cmaps = ['Purples', 'Reds']

for i, axi in enumerate(ax):
    axi.set_title(species_names[i])
    
    # plot coastlines with basemap
    m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                urcrnrlat=Y.max(), llcrnrlon=X.min(),
                urcrnrlon=X.max(), resolution='c', ax=axi)
    m.drawmapboundary(fill_color='#DDEEFF')
    m.drawcoastlines()
    m.drawcountries()
    
    # construct a spherical kernel density estimate of the distribution
    kde = KernelDensity(bandwidth=0.03, metric='haversine')
    kde.fit(np.radians(latlon[species == i]))

    # evaluate only on the land: -9999 indicates ocean
    Z = np.full(land_mask.shape[0], -9999.0)
    Z[land_mask] = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    axi.contourf(X, Y, Z, levels=levels, cmap=cmaps[i])

In [3]:
from sklearn.neighbors import KernelDensity
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import LeaveOneOut

acc_vals = accidents_data[['Latitude', 'Longitude']].values

In [4]:
xgrid = accidents_data['Latitude'].values
ygrid = accidents_data['Longitude'].values

In [ ]:
len(xgrid), len(ygrid)

(706947, 706947)

In [ ]:
mesh_x, mesh_y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])

stack_xy = np.radians(np.vstack([mesh_y.ravel(), mesh_x.ravel()]).T)

In [ ]:
bandwidths = np.linspace(0.1, 10, 100)
grid = GridSearchCV(KernelDensity(kernel='gaussian'), {'bandwidth': bandwidths}, cv=LeaveOneOut(len(acc_vals)))
grid.fit(acc_vals)

print('Best bandwidth: ', grid.best_params_)

In [ ]:
coord_vals = accidents_data[['Latitude', 'Longitude']].sample(10000).values

In [ ]:
from mpl_toolkits.basemap import Basemap, cm, maskoceans
from matplotlib import path
from sklearn.datasets.species_distributions import construct_grids
from sklearn.datasets import fetch_species_distributions

def mask_water(points, polys):
    """
    This method masks off the water (where data will be unreliable).
    """
    result = []
    for i, poly in enumerate(polys):
        if i == 0:
            mask = path.Path(poly).contains_points(points)
        else:
            mask = mask | path.Path(poly).contains_points(points)
    return np.array(mask)

def map_data(data, res=.2, dotsize=2, colors=['red', 'yellow', 'blue'], _kds=None):
    
    min_lat = data['Latitude'].min()
    max_lat = data['Latitude'].max()
    min_lon = data['Longitude'].min()
    max_lon = data['Longitude'].max()

    area = 0.1

    fig, axes = plt.subplots(1, 2, figsize=(20,20))
    fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)
    accident_severity = ['Major', 'Medium', 'Minor']
    
    for i, axis in enumerate(axes):
    
        map = Basemap(projection='merc', lat_0 = np.mean([min_lat, max_lat]), lon_0 = np.mean([min_lon, max_lon]),
            resolution = 'l', area_thresh = 0.1,
            llcrnrlon=min_lon - area, llcrnrlat=min_lat - area,
            urcrnrlon=max_lon + area, urcrnrlat=max_lat + area,
            epsg=5520)

        map.arcgisimage(service='World_Street_Map', xpixels = 200, verbose= True)

        #for c in data['Accident_Severity'].unique():
        #    lon = data[data['Accident_Severity'] == c]['Longitude'].values
        #    lat = data[data['Accident_Severity'] == c]['Latitude'].values
        #    x,y = map(lon, lat)
        #    map.plot(x, y, 'bo', markersize=dotsize, color=colors[c-1])

        model = KernelDensity(bandwidth=3., metric='haversine', kernel='gaussian', algorithm='ball_tree')
        model.fit(np.radians(data[data['Accident_Severity'] == i + 1][['Latitude','Longitude']]))

        x = np.arange(min_lat, max_lat, res)
        y = np.arange(min_lon, max_lon, res)
        x_grid, y_grid = np.meshgrid(x, y)
        elements_cnt = len(x_grid) * len(x_grid[0, :])

        x_unraveled = x_grid.reshape([elements_cnt, 1])
        y_unraveled = y_grid.reshape([elements_cnt, 1])
        data_to_eval = np.hstack([x_unraveled, y_unraveled])
        
        density = np.exp(model.score_samples(data_to_eval))#-9999 + np.zeros(data_to_eval.shape[0])
        #density = np.exp(model.score_samples(np.radians(data[data['Accident_Severity'] == i + 1][['Latitude','Longitude']])))
        # Mask water
        x, y = map(data_to_eval[:,1], data_to_eval[:,0])
        loc = np.c_[x, y]
        polys = [p.boundary for p in map.landpolygons]
        on_land = mask_water(loc, polys) 
        #density[~on_land] = np.exp(model.score_samples(data_to_eval))

        #Make predictions using appropriate model. 
        #density = maskoceans(x_grid, y_grid, density)
        density = density.reshape(x_grid.shape)
        #density = maskoceans(x_grid, y_grid, density)
        axis.contourf(y_grid, x_grid, density, levels=np.linspace(0, density.max(), 25), cmap='Purples')

In [ ]:
# Split the training data by label.

# For each set, fit a KDE to obtain a generative model of the data. This allows you for any observation xx and label yy to compute a likelihood P(x | y)P(x | y).

# From the number of examples of each class in the training set, compute the class prior, P(y)P(y).

# For an unknown point xx, the posterior probability for each class is P(y | x)∝P(x | y)P(y)P(y | x)∝P(x | y)P(y). The class which maximizes this posterior is the label assigned to the point.

In [ ]:
map_data(accidents_data)

In [ ]:
land_reference

In [ ]:
X

In [ ]:
Y

In [ ]:
np.array([d.decode('ascii').startswith('micro')
                  for d in data['train']['species']], dtype='int')

In [ ]:
# Author: Jake Vanderplas <jakevdp@cs.washington.edu>
#
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids
from sklearn.neighbors import KernelDensity

# if basemap is available, we'll use it.
# otherwise, we'll improvise later...
try:
    from mpl_toolkits.basemap import Basemap
    basemap = True
except ImportError:
    basemap = False

# Get matrices/arrays of species IDs and locations
data = fetch_species_distributions()
species_names = ['Bradypus Variegatus', 'Microryzomys Minutus']

Xtrain = np.vstack([data['train']['dd lat'],
                    data['train']['dd long']]).T
ytrain = np.array([d.decode('ascii').startswith('micro')
                  for d in data['train']['species']], dtype='int')
Xtrain *= np.pi / 180.  # Convert lat/long to radians

# Set up the data grid for the contour plot
xgrid, ygrid = construct_grids(data)
X, Y = np.meshgrid(xgrid[::5], ygrid[::5][::-1])
land_reference = data.coverages[6][::5, ::5]
land_mask = (land_reference > -9999).ravel()

xy = np.vstack([Y.ravel(), X.ravel()]).T
#xy = xy[land_mask]
xy *= np.pi / 180.

# Plot map of South America with distributions of each species
fig = plt.figure()
fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)

for i in range(2):
    plt.subplot(1, 2, i + 1)

    # construct a kernel density estimate of the distribution
    print(" - computing KDE in spherical coordinates")
    kde = KernelDensity(bandwidth=0.04, metric='haversine',
                        kernel='gaussian', algorithm='ball_tree')
    kde.fit(Xtrain[ytrain == i])

    # evaluate only on the land: -9999 indicates ocean
    Z = -9999 + np.zeros(land_mask.shape[0])
    #Z[land_mask] 
    Z = np.exp(kde.score_samples(xy))
    Z = Z.reshape(X.shape)

    # plot contours of the density
    levels = np.linspace(0, Z.max(), 25)
    plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)

    if basemap:
        print(" - plot coastlines using basemap")
        m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                    urcrnrlat=Y.max(), llcrnrlon=X.min(),
                    urcrnrlon=X.max(), resolution='c')
        m.drawcoastlines()
        m.drawcountries()
    else:
        print(" - plot coastlines from coverage")
        plt.contour(X, Y, land_reference,
                    levels=[-9999], colors="k",
                    linestyles="solid")
        plt.xticks([])
        plt.yticks([])

    plt.title(species_names[i])


In [ ]:
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids
from mpl_toolkits.basemap import Basemap, cm, maskoceans

# Get matrices/arrays of species IDs and locations
latlon = accidents_data[['Latitude','Longitude']]#.as_matrix()
res = 1

# Create two side-by-side plots
fig, ax = plt.subplots(1, 2)
axi = ax[0]

fig.subplots_adjust(left=0.05, right=0.95, wspace=0.05)

x = np.arange(min(accidents_data['Latitude']), max(accidents_data['Latitude']), res)
y = np.arange(min(accidents_data['Longitude']), max(accidents_data['Longitude']), res)

X, Y = np.meshgrid(x, y)

numel = len(X) * len(X[0, :])

unraveled_x = X.reshape([numel, 1])
unraveled_y = Y.reshape([numel, 1])
data_to_eval = np.hstack([unraveled_x, unraveled_y])

# construct a spherical kernel density estimate of the distribution
kde = KernelDensity(bandwidth=0.03, metric='haversine')
kde.fit(np.radians(latlon))

m = Basemap(projection='cyl', llcrnrlat=Y.min(),
            urcrnrlat=Y.max(), llcrnrlon=X.min(),
            urcrnrlon=X.max(), resolution='c', ax=axi)

m.drawcoastlines()
# Mask water
#x, y = m(data_to_eval[:,1], data_to_eval[:,0])
#loc = np.c_[x, y]
#polys = [p.boundary for p in m.landpolygons]
#on_land = mask_water(loc, polys) 

m.drawmapboundary(fill_color='#DDEEFF')
m.drawcountries()

# evaluate only on the land: -9999 indicates ocean
density = np.exp(kde.score_samples(latlon))
#density[~on_land] = np.exp(kde.score_samples(latlon))
#density = density.reshape(X.shape)

data = maskoceans(X, Y, density)
# plot contours of the density
levels = np.linspace(0, density.max(), 25)
axi.contourf(X, Y, data, levels=levels, cmap='Purples')

In [ ]:
from mpl_toolkits.basemap import Basemap
import numpy as np
import matplotlib.pyplot as plt

def points_in_polys(points, polys):
    result = []
    for poly in polys:
        mask = nx.points_inside_poly(points, poly)
        result.extend(points[mask])
        points = points[~mask]
    return np.array(result)

points = np.random.randint(0, 90, size=(100000, 2))
m = Basemap(projection='moll',lon_0=0,resolution='c')
m.drawcoastlines()
m.fillcontinents(color='coral',lake_color='aqua')

x, y = m(points[:,0], points[:,1])
loc = np.c_[x, y]
polys = [p.boundary for p in m.landpolygons]
land_loc = points_in_polys(loc, polys)
m.plot(land_loc[:, 0], land_loc[:, 1],'ro')


In [ ]:
kde.score_samples(xy)

In [ ]:
# In Kernel Density estimation, the bandwidth parameter controls the variance / bias tradeoff. 
# Therefore, we would want to pick a hyperparameter that is not too narrow and not too wide.
# For choosing the optimal value, we test a set of bandwidths through cross validation.
bandwidths = [300, 1000]#[100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
data = np.radians(coordinates[severity == 1])
params = {'bandwidth': bandwidths, 'metric': ['haversine']}
grid = GridSearchCV(KernelDensity(kernel='gaussian'), params, cv=LeaveOneOut(len(data)))
grid.fit(data)
print('Model fit. \nBest bandwidth value: %s' % grid.best_params_)

#kde = KernelDensity(bandwidth=0.03, metric='haversine')
#kde.fit(np.radians(coordinates[severity == 1]))

In [ ]:
from pylab import *
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.neighbors.kde import KernelDensity
from matplotlib import path
import random
from mpl_toolkits.basemap import Basemap 
import numpy as np
import pandas as pd




def makeNearestNeighborsDensityPlot(geolocated, res = .2):
    #Filter for events with locations. 

    model = KernelDensity(kernel='gaussian', bandwidth = 3).fit(geolocated[['Latitude','Longitude']])
    
    #Create a grid of points at which to predict. 
    min_lat, max_lat = min(geolocated['Latitude']), max(geolocated['Latitude'])
    min_lon, max_lon = min(geolocated['Longitude']), max(geolocated['Longitude'])
    
    x = np.arange(min_lat, max_lat, res)
    y = np.arange(min_lon, max_lon, res)
    X, Y = meshgrid(x, y)
    numel = len(X) * len(X[0, :])
    
    unraveled_x = X.reshape([numel, 1])
    unraveled_y = Y.reshape([numel, 1])
    data_to_eval = np.hstack([unraveled_x, unraveled_y])
    palettes = ['Purples','Blues']
    
    #Make predictions using appropriate model. 
    density = np.exp(model.score_samples(data_to_eval))

    #Make map. 
    figure(figsize = [20, 10])    
    m = Basemap(llcrnrlat = min_lat, urcrnrlat = max_lat, llcrnrlon = min_lon, urcrnrlon=max_lon, resolution='l', fix_aspect = False)

    # Mask water
    #m.drawcoastlines()
    #x, y = m(data_to_eval[:,1], data_to_eval[:,0])
    #loc = np.c_[x, y]
    #polys = [p.boundary for p in m.landpolygons]
    #on_land = mask_water(loc, polys) 
    #density[~on_land] = (color_min + color_max) / 2
    
    density = density.reshape(X.shape)
    contourf(Y, X, density, levels=np.linspace(0, density.max(), 25), cmap=palettes[0])
    m.drawcoastlines(linewidth = 2)
    m.drawcountries(linewidth=2)
    m.drawstates(linewidth = 2)
    colorbar()
    set_cmap(cmap)

if __name__ == '__main__':
    print("Making plot without normalizing!")
    makeNearestNeighborsDensityPlot(accidents_data[['Latitude','Longitude','Accident_Severity']])

In [ ]:
# Authors: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#          Jake Vanderplas <vanderplas@astro.washington.edu>
#
# License: BSD 3 clause

from __future__ import print_function

from time import time

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets.base import Bunch
from sklearn.datasets import fetch_species_distributions
from sklearn.datasets.species_distributions import construct_grids
from sklearn import svm, metrics

# if basemap is available, we'll use it.
# otherwise, we'll improvise later...
try:
    from mpl_toolkits.basemap import Basemap
    basemap = True
except ImportError:
    basemap = False

print(__doc__)


def create_species_bunch(species_name, train, test, coverages, xgrid, ygrid):
    """Create a bunch with information about a particular organism

    This will use the test/train record arrays to extract the
    data specific to the given species name.
    """
    bunch = Bunch(name=' '.join(species_name.split("_")[:2]))
    species_name = species_name.encode('ascii')
    points = dict(test=test, train=train)

    for label, pts in points.items():
        # choose points associated with the desired species
        pts = pts[pts['species'] == species_name]
        bunch['pts_%s' % label] = pts

        # determine coverage values for each of the training & testing points
        ix = np.searchsorted(xgrid, pts['dd long'])
        iy = np.searchsorted(ygrid, pts['dd lat'])
        bunch['cov_%s' % label] = coverages[:, -iy, ix].T

    return bunch


def plot_species_distribution(species=("bradypus_variegatus_0",
                                       "microryzomys_minutus_0")):
    """
    Plot the species distribution.
    """
    if len(species) > 2:
        print("Note: when more than two species are provided,"
              " only the first two will be used")

    t0 = time()

    # Load the compressed data
    data = fetch_species_distributions()

    # Set up the data grid
    xgrid, ygrid = construct_grids(data)

    # The grid in x,y coordinates
    X, Y = np.meshgrid(xgrid, ygrid[::-1])

    # create a bunch for each species
    BV_bunch = create_species_bunch(species[0],
                                    data.train, data.test,
                                    data.coverages, xgrid, ygrid)
    MM_bunch = create_species_bunch(species[1],
                                    data.train, data.test,
                                    data.coverages, xgrid, ygrid)

    # background points (grid coordinates) for evaluation
    np.random.seed(13)
    background_points = np.c_[np.random.randint(low=0, high=data.Ny,
                                                size=10000),
                              np.random.randint(low=0, high=data.Nx,
                                                size=10000)].T

    # We'll make use of the fact that coverages[6] has measurements at all
    # land points.  This will help us decide between land and water.
    land_reference = data.coverages[6]

    # Fit, predict, and plot for each species.
    for i, species in enumerate([BV_bunch, MM_bunch]):
        print("_" * 80)
        print("Modeling distribution of species '%s'" % species.name)

        # Standardize features
        mean = species.cov_train.mean(axis=0)
        std = species.cov_train.std(axis=0)
        train_cover_std = (species.cov_train - mean) / std

        # Fit OneClassSVM
        print(" - fit OneClassSVM ... ", end='')
        clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.5)
        clf.fit(train_cover_std)
        print("done.")

        # Plot map of South America
        plt.subplot(1, 2, i + 1)
        if basemap:
            print(" - plot coastlines using basemap")
            m = Basemap(projection='cyl', llcrnrlat=Y.min(),
                        urcrnrlat=Y.max(), llcrnrlon=X.min(),
                        urcrnrlon=X.max(), resolution='c')
            m.drawcoastlines()
            m.drawcountries()
        else:
            print(" - plot coastlines from coverage")
            plt.contour(X, Y, land_reference,
                        levels=[-9999], colors="k",
                        linestyles="solid")
            plt.xticks([])
            plt.yticks([])

        print(" - predict species distribution")

        # Predict species distribution using the training data
        Z = np.ones((data.Ny, data.Nx), dtype=np.float64)

        # We'll predict only for the land points.
        idx = np.where(land_reference > -9999)
        coverages_land = data.coverages[:, idx[0], idx[1]].T

        pred = clf.decision_function((coverages_land - mean) / std)[:, 0]
        Z *= pred.min()
        Z[idx[0], idx[1]] = pred

        levels = np.linspace(Z.min(), Z.max(), 25)
        Z[land_reference == -9999] = -9999

        # plot contours of the prediction
        plt.contourf(X, Y, Z, levels=levels, cmap=plt.cm.Reds)
        plt.colorbar(format='%.2f')

        # scatter training/testing points
        plt.scatter(species.pts_train['dd long'], species.pts_train['dd lat'],
                    s=2 ** 2, c='black',
                    marker='^', label='train')
        plt.scatter(species.pts_test['dd long'], species.pts_test['dd lat'],
                    s=2 ** 2, c='black',
                    marker='x', label='test')
        plt.legend()
        plt.title(species.name)
        plt.axis('equal')

        # Compute AUC with regards to background points
        pred_background = Z[background_points[0], background_points[1]]
        pred_test = clf.decision_function((species.cov_test - mean)
                                          / std)[:, 0]
        scores = np.r_[pred_test, pred_background]
        y = np.r_[np.ones(pred_test.shape), np.zeros(pred_background.shape)]
        fpr, tpr, thresholds = metrics.roc_curve(y, scores)
        roc_auc = metrics.auc(fpr, tpr)
        plt.text(-35, -70, "AUC: %.3f" % roc_auc, ha="right")
        print("\n Area under the ROC curve : %f" % roc_auc)

    print("\ntime elapsed: %.2fs" % (time() - t0))


plot_species_distribution()
plt.show()

In [ ]:
# Build a predictive model to infer the coordinates from the data
import xgboost as xgb
from sklearn.model_selection import train_test_split

y_lat = accidents['Latitude'].values
y_lon = accidents['Longitude'].values

x_train_lat, x_test_lat, y_train_lat, y_test_lat = train_test_split(
    accidents.drop(['Accident_Index', 'Longitude','Latitude'], axis=1).astype(float), 
    y_lat, test_size=0.3, random_state=54)

x_train_lon, x_test_lon, y_train_lon, y_test_lon = train_test_split(
    accidents.drop(['Accident_Index', 'Longitude','Latitude'], axis=1).astype(float), 
    y_lon, test_size=0.3, random_state=54)

d_train_lat = xgb.DMatrix(x_train_lat, y_train_lat)
d_train_lon = xgb.DMatrix(x_train_lon, y_train_lon)

d_test_lat = xgb.DMatrix(x_test_lat)
d_test_lon = xgb.DMatrix(x_test_lon)

In [ ]:
param = {'max_depth':2, 'eta':0.3, 'silent':0, 'objective':'reg:linear', 'eval_metric':'rmse', 'seed':12}
model_lat = xgb.train(param, d_train_lat, 10)
prediction_lat = model_lat.predict(d_test_lat)
prediction_